## Задача
- [x] Сгенерировать 10 000 пользователей с помощью UUID4;
- [x] С вероятностью в 10% назначить пользователям факт совершения некоторого действия;
- [x] Распределить пользователей 10 000 раз по контрольной и тестовой группе. Для каждой итерации новая соль;
- [x] Для каждой итерации определить значимость отличия конверсии в исходное действие с p_value = 0.05;
- [ ] Построить распределение полученных 10 000 p_value на графике. Какое распределение получилось? 
- [ ] Проверить соответствие полученного распределения с равномерным с помощью теста Колмогорова-Смирнова;
- [ ] Сделать выводы;
 

## Импорты и функции

In [1]:
import uuid
import hashlib
import warnings
import sqlite3
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest
from tqdm.auto import tqdm

In [2]:
warnings.filterwarnings('ignore')
connect = sqlite3.connect('../db.sqlite')

In [3]:
# Параметры симуляции 
NUM_USERS = 10_000
NUM_EXPERIMENTS = 10_000
ACTION_PROBABILITY = 0.1
ALPHA = 0.05
CREATE_DATA = 'load' # 'generate

In [4]:
def generate_users_with_actions(num_users: int = None, action_probability: int = None) -> pd.DataFrame:
    """Генерирует популяцию с заданным количеством пользователей и заданной вероятностью конверсии в целевое действие"""

    np.random.seed(42)

    users = [str(uuid.uuid4()) for _ in range(num_users)]
    actions = np.random.binomial(1, action_probability, size=num_users)
    data = {}
    for key, val in zip(users, actions):
        data[key] = val

    result = pd.DataFrame(data=list(data.items()), columns=['user', 'action'])

    return result

In [5]:
def split_users(user_id, salt: str) -> int:
    """Распределяет пользователей по 2м группам АБ-теста"""

    hash_object = hashlib.md5((user_id + salt).encode())
    hash_hex = hash_object.hexdigest()
    hash_int = int(hash_hex, 16)

    group = ''
    if hash_int % 100 <= 50:
        group = 'control'
    else:
        group = 'test'

    return group

In [6]:
def create_and_estimate_ab(data: pd.DataFrame, salt: int, alpha: int) -> pd.DataFrame:
    """Проводит АБ и возвращает результаты"""

    data = data.copy()
    data['group'] = data.apply(lambda row: split_users(user_id=row['user'], salt=str(salt)), axis=1)
    data = data.pivot_table(index=['group'], columns='action', values='user', aggfunc='count',
                            margins=True, margins_name='total')

    separation = {
        'control_successes': data.loc['control', 1],
        'control_total': int(data.loc['control', 'total']),
        'control_cr': data.loc['control', 1] / data.loc['control', 'total'],
        'test_successes': data.loc['test', 1],
        'test_total': data.loc['test', 'total'],
        'test_cr': data.loc['test', 1] / data.loc['test', 'total'],
    }

    z_score, p_value = proportions_ztest(
        count=[separation['control_successes'], separation['test_successes']],
        nobs=[separation['control_total'], separation['test_total']],
        alternative='two-sided')

    if p_value < alpha:
        h_0 = 'reject'
    else:
        h_0 = 'not_reject'

    results_ab = {
        'h_0': h_0,
        'p_value': p_value,
        'z_score': z_score,
        'control_cr': separation['control_cr'],
        'test_cr': separation['test_cr'],
        'control_total': separation['control_total'],
        'control_successes': separation['control_successes'],
        'test_total': separation['test_total'],
        'test_successes': separation['test_successes']
    }

    df_row = pd.DataFrame(results_ab, index=[0])

    del data

    return df_row

## Готовим данные 

In [7]:
# Для сбора результатов
results = pd.DataFrame(
    columns=['h_0', 'p_value', 'z_score', 'control_cr', 'test_cr', 'control_total',
             'control_successes', 'test_total', 'test_successes'])

# Создаем популяцию 
population = generate_users_with_actions(num_users=NUM_USERS, action_probability=ACTION_PROBABILITY)

if CREATE_DATA == 'generate':
    for i in tqdm(range(NUM_EXPERIMENTS)):
        exp = create_and_estimate_ab(data=population, salt=i, alpha=ALPHA)
        results = pd.concat([results, exp], ignore_index=True)
    
    # Мини-фикс на тип данных, чтобы в итоговом DF инты были интами :) 
    results[['control_total', 'control_successes', 'test_total', 'test_successes']] = results[
        ['control_total', 'control_successes', 'test_total', 'test_successes']].astype(int)
    
    # Сохраним в БД, чтобы не гонять тесты каждый раз.
    results.to_sql(name='intensive_syntetic_ab_test', con=connect, if_exists='replace')

## Анализируем данные

In [8]:
if CREATE_DATA == 'load':
    df = pd.read_sql('SELECT * FROM intensive_syntetic_ab_test', con=connect)

In [9]:
df

,index,h_0,p_value,z_score,control_cr,test_cr,control_total,control_successes,test_total,test_successes
0,0,not_reject,0.311921,-1.011200,0.093165,0.099127,5077,473,4923,488
1,1,reject,0.000448,3.510022,0.106131,0.085431,5154,547,4846,414
2,2,not_reject,0.081646,-1.741216,0.091138,0.101407,5168,471,4832,490
3,3,not_reject,0.213864,1.243010,0.099687,0.092358,5106,509,4894,452
4,4,not_reject,0.963812,-0.045370,0.095969,0.096236,5085,488,4915,473
...,...,...,...,...,...,...,...,...,...,...
9995,9995,not_reject,0.855663,0.181897,0.096623,0.095551,5123,495,4877,466
9996,9996,not_reject,0.113019,1.584767,0.100597,0.091249,5189,522,4811,439
9997,9997,not_reject,0.686664,0.403386,0.097280,0.094902,5037,490,4963,471
9998,9998,reject,0.007764,-2.662170,0.088363,0.104057,5070,448,4930,513
